### 1. Explain the difference between “__new__” and “__init__” in Python class instantiation, and provide a scenario where overriding new is necessary (e.g., for implementing a singleton pattern).

- `__new__` is a **static method** responsible for **creating a new instance** of a class. → useful for singleton for example.
- It is called **before** `__init__`.
- `__init__` is an **instance method** that initializes the newly created object.

In [5]:
# singleton

class Singleton:
	_instance = None
	def __new__(cls, *args, **kwargs):
		if cls._instance is None:
			cls._instance = super().__new__(cls)
		return cls._instance
		
	def __init__(self, value, *args, **kwargs):
		if not hasattr(self, "_initialized"):
			self.value = value
			self._initialized = True
			
			
# usage
s1 = Singleton(10)
s2 = Singleton(20)

print(s1.value)    
print(s2.value)    

10
10


---

### 2. What are metaclasses in Python? Write a custom metaclass that automatically registers all subclasses and enforces that every class has a specific method (e.g., validate()). Discuss real-world uses, such as in ORM frameworks like Django.

metaclass → classes of classes

- **a metaclass controls class creation and modification**.

1. **Instance** → an object of a class.
2. **Class** → defines how instances behave.
3. **Metaclass** → defines how classes behave.
- In Python, **everything is an object**.
- Classes themselves are **instances of a metaclass**.

• The default metaclass is `type`:

```python
class MyClass:
	pass
	
print(type(MyClass))   # <class 'type'>
```


**Key Idea:** Metaclasses let you **intercept class creation** and customize it.

A metaclass usually overrides **`__new__`** or **`__init__`**

In [ ]:
class MyMeta(type):
	def __new__(cls, name, bases, namespace):
		print(f"Creating class {name}")
		return super().__new__(cls, name, bases, namespace)
		
class Person(metaclass=MyMeta):
	def greet(self):
		print("Hello")
		
# Output: Creating class Person -> when the class is defined (this is shown when we just define the class!)

Creating class Person


`MyMeta.__new__` is called **when the class is defined**, not when instances are created.

In [ ]:
# another example of meta class
class RegistryMeta(type):
    registry = []

    def __new__(cls, name, bases, namespace):
        # Enforce 'validate' method
        if 'validate' not in namespace:
            raise TypeError(f"Class {name} must implement a 'validate()' method")
        
        # Create the class
        new_class = super().__new__(cls, name, bases, namespace)
        
        # Register the class
        cls.registry.append(new_class)
        return new_class

# Base class
class Base(metaclass=RegistryMeta):
    def validate(self):
        pass

# Subclasses
class User(Base):
    def validate(self):
        print("Validating user")

class Product(Base):
    def validate(self):
        print("Validating product")

print(RegistryMeta.registry)

[<class '__main__.Base'>, <class '__main__.User'>, <class '__main__.Product'>]


Singleton Pattern with metaclass:

In [7]:
class SingletonMeta(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class Logger(metaclass=SingletonMeta):
    pass

a = Logger()
b = Logger()
print(a is b)  # True

True


at first it is confusing. but just consider the order of execution.
Ultra-Short Rule (Memorize This)

class → metaclass `__new__()` → metaclass `__call__` → class `__new__` → class `__init__`

#### Real-World Uses of Metaclasses

1. ORM Frameworks (Django, SQLAlchemy)

- Django’s ModelBase metaclass scans model fields and prepares database mappings.
- You don’t have to write SQL manually; metaclass handles it.

2. Plugin Systems

- Automatically register subclasses, so plugins are discoverable without extra boilerplate.

3. Singleton Pattern (as i wrote above)

4. Enforcing Class Interfaces

Automatically ensure certain methods or attributes(like ORM fields) exist on all subclasses (like our validate() example).

---

### 3. Describe Python's Global Interpreter Lock (GIL). How does it impact multithreading, and when would you use multiprocessing instead of threading? Provide an example where the GIL causes unexpected performance issues.

GIL -> a mutex inside the CPython interpreter -> ensures only one thread executes Python bytecode at any given time.

what is mutex? -> mutual exclusion : a synchronization primitive -> used in concurrent programming -> ensure that only one thread or process at a time can access a shared resource or execute a critical section of code.

=> At any moment, at most one owner can hold a mutex.

In [ ]:
import threading

lock = threading.Lock()   # mutex
counter = 0

def increment():
    global counter
    with lock:              # acquire
        counter += 1        # critical section
                            # release automatically

threads = [threading.Thread(target=increment) for _ in range(1000)]
for t in threads:
    t.start()
for t in threads:
    t.join()

print(counter)

1000


why GIL exisits? -> because CPython’s core data structures : are not thread-safe.

The GIL:

- Protects reference counting
- Avoids fine-grained locking across every object
- Simplifies the interpreter design
- Improves single-threaded performance

The GIL DOES lock: Execution of Python bytecode

the question is : why not use OS for managing threads? -> answer: 

The OS does not know the rules of Python objects:

- Python objects like lists, dicts, sets have internal structure

- Reference counts track how many references exist for memory management

- Operations like my_list.append(1) are not atomic — they involve multiple steps:

1. Check memory

2. Update internal array

3. Update reference counts

If two threads run these steps at the same time:

- OS does not know that Python needs these steps to be atomic

- Result: memory corruption, crashes, incorrect data






NEXT QUESTION: What would remove the GIL???

To remove the GIL, you need:

A different Python implementation that is thread-safe:

- Jython (runs on Java, no GIL)

- IronPython (runs on .NET, no GIL)

- Or rewrite CPython internals so that all objects are thread-safe (very hard, huge overhead)

#### when would you use multiprocessing instead of threading?

| Concept    | Threading                         | Multiprocessing                                        |
| ---------- | --------------------------------- | ------------------------------------------------------ |
| Uses       | Threads in **same process**       | Separate **processes**                                 |
| Memory     | Shared                            | Separate memory (do not share globals by default)      |
| GIL impact | Still limited by GIL for CPU code | Each process has **its own Python interpreter + GIL**  |
| Overhead   | Low                               | Higher (process creation, inter-process communication) |


Use threading if your program is I/O-bound, meaning it spends a lot of time waiting for:

- Network requests (API calls, sockets)

- Disk operations (file read/write)

- Database queries

In [12]:
import threading
import requests

urls = ["http://google.com"] * 10
def fetch(url):
    requests.get(url)
    
    
threads = [threading.Thread(target=fetch, args=(u,)) for u in urls]
[t.start() for t in threads]
[t.join() for t in threads]

[None, None, None, None, None, None, None, None, None, None]

In [13]:
threads

[<Thread(Thread-1020 (fetch), stopped 140440120583872)>,
 <Thread(Thread-1021 (fetch), stopped 140440128976576)>,
 <Thread(Thread-1022 (fetch), stopped 140440876586688)>,
 <Thread(Thread-1023 (fetch), stopped 140440884979392)>,
 <Thread(Thread-1024 (fetch), stopped 140440910157504)>,
 <Thread(Thread-1025 (fetch), stopped 140440901764800)>,
 <Thread(Thread-1026 (fetch), stopped 140440112191168)>,
 <Thread(Thread-1027 (fetch), stopped 140440103798464)>,
 <Thread(Thread-1028 (fetch), stopped 140440095405760)>,
 <Thread(Thread-1029 (fetch), stopped 140440087013056)>]

we used threads because it was i/o task.

Use multiprocessing if your program is CPU-bound, meaning it does heavy computations like:

- Number crunching

- Simulations

- Backtesting large datasets

In [17]:
from multiprocessing import Pool


def compute(n):
    return sum(i*i for i in range(n))

with Pool(processes=4) as pool:
    results = pool.map(compute, [10**7]*4)
    
results

[333333283333335000000,
 333333283333335000000,
 333333283333335000000,
 333333283333335000000]

- Each process has its own interpreter and GIL

- Computation runs in true parallel on multiple CPU cores

#### Provide an example where the GIL causes unexpected performance issues.

In [26]:
# Example: CPU-bound computation with threads
import threading
import time

# CPU-heavy task
def count(n):
    total = 0
    for i in range(n):
        total += i*i
    return total

N = 100_000_000  # large number for noticeable computation

# Using threads
threads = []
start = time.time()
for _ in range(4):  # 4 threads
    t = threading.Thread(target=count, args=(N,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("Threading took:", time.time() - start)

Threading took: 14.387580394744873


In [24]:
import time

# CPU-heavy task
def count(n):
    total = 0
    for i in range(n):
        total += i*i
    return total

N = 100_000_000  # large number for noticeable computation

start = time.time()

result = count(N)


print("without multi-threading took:", time.time() - start)

without multi-threading took: 3.321453332901001


our expectation was: 4 threads → should be ~4x faster on 4 cores

why? -> GIL prevents threads from running Python bytecode in parallel

##### using multiprocessing in this case

In [28]:
from multiprocessing import Pool
import time

def count(n):
    total = 0
    for i in range(n):
        total += i*i
    return total

N = 100_000_000

with Pool(4) as p:  # 4 processes
    start = time.time()
    p.map(count, [N]*4)

print("Multiprocessing took:", time.time() - start)


Multiprocessing took: 4.8356335163116455
